### Matrix Factorization of Correlation Matrix

Simple baseline factorizing the correlation matrix.

In [1]:
import sys
sys.path.append('../')

In [2]:
from utils.returns_data_class import ReturnsData
data = ReturnsData(daily_returns_path="../Data/returns_df_611.csv",
                   extras_path="../Data/historical_stocks.csv",)

In [3]:
from models.embedding_models import MatrixFactorization
# correlations = torch.tensor(data.returns_df.corr().values)
correlations = MatrixFactorization.get_correlation_matrix(X=data.daily_returns_df.T.values, scaled=False)
# correlations = MatrixFactorization.apply_IQR_mask(correlations, percentiles=(0.25,0.75))
# correlations = MatrixFactorization.apply_random_mask(correlations, percentage=0.9)

In [4]:
model, losses, learning_rates = MatrixFactorization.train_MF_model(
    n_time_series=len(data.tickers),
    similarity_matrix=correlations,
    learning_rate=0.2,
    epochs=300,
    regularization_loss_weight=0.1,
    pairwise_loss_weight=0.01,
    verbose=True,
)

100%|██████████| 300/300 [00:02<00:00, 145.04it/s]


In [5]:

MatrixFactorization.plot_embedding_training(losses, learning_rates, verbose=True)

Final pairwise_loss: 31.934613429025298


In [6]:
from utils.sector_classification import get_sector_score

get_sector_score(model.embeddings.weight.detach().numpy(), sectors=data.sectors, top_k_accuracy=True)

Precision Score: 0.72
Recall Score: 0.68
F1 Score: 0.69
Accuracy Score: 0.68
Accuracy Score Top-3: 0.88


In [7]:
from utils.visualisation_functions import pca_plot_from_embeddings

pca_plot_from_embeddings(
    embedding_matrix=model.embeddings.weight.detach().numpy(),
    sectors=data.sectors,
    tickers=data.tickers,
    industries=data.industries,
    names=data.names,
    dimensions=2,
    reduced=True,
    method="PCA",
    return_df=False,
    rand_state=None,
)

In [30]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
# classifier = KNeighborsClassifier(n_neighbors=1, metric='euclidean')

# embeddings = model.embeddings.weight.detach().numpy()
embeddings = data.daily_returns_df.values.T
train_size = int(len(data.sectors)*0.7)
X_train = embeddings[:train_size, :]
X_test = embeddings[train_size:, :]
y_train = data.sectors[:train_size]
y_test = data.sectors[train_size:]

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from utils.ucr_helpers import evaluate_resampling_UCR
report, _, _ = evaluate_resampling_UCR(
    X_train,
    X_test,
    y_train,
    y_test,
    # classifier=SVC(kernel="rbf"),
    # classifier=LogisticRegression(),
    classifier=KNeighborsClassifier(n_neighbors=1),
    # classifier=MLPClassifier(),
    n_resamples=20,
    verbose=True,
    scale=False,
    over_sampling=True
)
report

 21%|██        | 4/19 [00:00<00:00, 32.50it/s]/opt/homebrew/Caskroom/miniforge/base/envs/phd/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/homebrew/Caskroom/miniforge/base/envs/phd/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/homebrew/Caskroom/miniforge/base/envs/phd/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

100%|██████████| 19/19 [00:00<00:00, 21.41it/s]


{'precision': 0.6939196754739662,
 'recall': 0.6396739130434781,
 'f1-score': 0.6363890733607594,
 'accuracy': 0.6396739130434782}